## File 01 - User-level Preprocessed Data Output

In this file, we create the preprocessed user-level data with a few features.

Outputs:
- The preprocessed data file is output to `processed_data/preprocessed_01.parquet`.
- Additionally, the raw data is filtered on the preprocessed data user ids: `processed_data/month_01_filtered.parquet`, `processed_data/month_02_filtered.parquet`.

And then in script 02, we add additional features to the preprocessed data table for every row.

### Set up Spark session and data schema

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

schema = "`event_time` TIMESTAMP,`event_type` STRING,`product_id` INT,`category_id` BIGINT,`category_code` STRING,`brand` STRING,`price` FLOAT,`user_id` INT,`user_session` STRING"
#ddl_schema = T._parse_datatype_string(schema)

CPU times: user 168 ms, sys: 136 ms, total: 304 ms
Wall time: 4.88 s


### Read in dataframes

In [2]:
%%time
df1 = spark.read.schema(schema).csv("/project/ds5559/group12/raw_data/2020-01.csv")
df2 = spark.read.schema(schema).csv("/project/ds5559/group12/raw_data/2020-02.csv")

CPU times: user 1.53 ms, sys: 2.38 ms, total: 3.91 ms
Wall time: 1.69 s


### Limit number of records in dataframes

We can limit each dataframe to a smaller subset. Notably, the dataframe is arranged by time, so this is how the subset will be biased.

In [3]:
# df1=df1.limit(100000)
df1.createOrReplaceTempView("m1")

# df2=df11.limit(100000)
df2.createOrReplaceTempView("m2")

### Create a transformed table containing elements of interest for our model

We propose a basic table format (see https://docs.google.com/document/d/1NG4KGticBXn0D3PL5_zMxLV2Pr7A8PQtLcasxCOd1nA/edit).

Every row is a user_id who exists in M1 and may or may not exist in M2.
Columns include:
- `user_id` (ID)
- `m2_total_spend` (sum among all purchase events, NB: month 2. Will be 0 is user does not exist in month 2 or makes no purchase events)
- `m1_total_spend` (sum among all purchase events)
- `m1_user_sessions` (count of distinct user sessions/browsing sessions)
- `m1_purchase_events` (count of distinct purchase events)

Additional rows may have been added below.


In [4]:
%%time
df = spark.sql("""SELECT

               /* ID */
               m1.user_id AS user_id,
               
               /* Output: Total spend in month 2; Will be 0 is user does not exist in month 2 or makes no purchase events */
               IFNULL(SUM(m2.price),0) AS T_total_spend,
               
               
               /* Total spend in month 1: we sum the price of any 'purchase' events */
               SUM(CAST(m1.event_type=='purchase' AS INT) * m1.price) AS total_spend,
               
               /* Total events in month 1: includes all event types (multiple per session) */
               COUNT(m1.event_type) AS total_events,
               
               /* Total purchase events in month 1: we sum the occurence of any 'purchase' events */
               /* SUM(CAST(m1.event_type=='purchase' AS INT)) AS purchase_events, */
               
               /* Total user sessions in month 1: we count all distinct user sessions */
               COUNT(DISTINCT m1.user_session) AS total_sessions
               
               FROM m1
            
            /* Note: This is a left join, so purchasers in month 2 must be in month 1 to be included in the output */
            LEFT JOIN 
            (
               SELECT * FROM m2
               WHERE event_type='purchase'
            ) m2
            
            ON m1.user_id=m2.user_id
            
            /* Prevent adding bad data where user_id is null */
            WHERE ISNULL(m1.user_id)<>1
            
            GROUP BY m1.user_id ORDER BY total_events DESC""")
df.show(5)

+---------+--------------------+--------------------+------------+--------------+
|  user_id|       T_total_spend|         total_spend|total_events|total_sessions|
+---------+--------------------+--------------------+------------+--------------+
|568782581|2.3885857005200386E8| 5.821735228239441E7|      926856|           532|
|582826305|1.9694101198085403E8|1.7104022296691895E7|      605784|           123|
|563599039|2.3802253637968063E8| 6.886928014480591E7|      442758|           108|
|568805468|  2.09597173788414E8|  3391851.5849990845|      425169|          4636|
|592727922|1.1673367678547668E8|   8990452.824829102|      423648|            78|
+---------+--------------------+--------------------+------------+--------------+
only showing top 5 rows

CPU times: user 4.07 ms, sys: 9.02 ms, total: 13.1 ms
Wall time: 1min 4s


#### Filter the overall dataset to a subset in order to make sure that the output from the feature addition script is under 3GB; then write to Parquet file

In [5]:
%%time

# We can control this level to make the preprocessed data under 3 GB
split_level = 0.05 # percentage

kept_df, garbage_df = df.randomSplit([split_level, 1-split_level], seed=42)
kept_df.write.mode("overwrite").parquet("./processed_data/preprocessed_01.parquet")
print(kept_df.count())
kept_df.show(5)

219080
+---------+-------------+-----------+------------+--------------+
|  user_id|T_total_spend|total_spend|total_events|total_sessions|
+---------+-------------+-----------+------------+--------------+
|405614124|          0.0|        0.0|           2|             2|
|485991194|          0.0|        0.0|           3|             1|
|496765250|          0.0|        0.0|           1|             1|
|501980918|          0.0|        0.0|        2141|            40|
|502621333|          0.0|        0.0|           5|             3|
+---------+-------------+-----------+------------+--------------+
only showing top 5 rows

CPU times: user 9.55 ms, sys: 99.1 ms, total: 109 ms
Wall time: 2min 40s


#### Write to CSV file

In [6]:
%%time
# Not necessary at this time, but this CSV can be written if desired
# kept_df.coalesce(1).write.option("header", "true").csv("./processed_data/temp_preprocessed_01.csv")

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.53 µs


#### Write the raw data, filtered on the appropriate user-ids, to files.

In [7]:
%%time
month_01_filtered = df1.join(kept_df,'user_id','leftsemi')
print(month_01_filtered.count())
month_01_filtered.show(5)

2807167
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|  user_id|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|        user_session|
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|405614124|2020-01-06 11:42:08|      view|   1004767|2232732093077520756|construction.tool...|samsung|232.77|18bc9e5e-d556-4cd...|
|405614124|2020-01-23 21:47:06|      view|   4804718|2232732079706079299|       sport.bicycle|  apple|321.41|7e972da4-7007-46d...|
|485991194|2020-01-06 18:35:24|      view|   1004767|2232732093077520756|construction.tool...|samsung|232.67|a7ac67d4-bec6-42b...|
|485991194|2020-01-06 18:41:44|      view|   1004767|2232732093077520756|construction.tool...|samsung|232.67|a7ac67d4-bec6-42b...|
|485991194|2020-01-06 18:42:02|      view|   1004767|2232732093077520756|co

In [8]:
# Probably not actually needed
%%time
month_02_filtered = df2.join(kept_df,'user_id','leftsemi')
print(month_02_filtered.count())
month_02_filtered.show(5)

1799982
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|  user_id|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|        user_session|
+---------+-------------------+----------+----------+-------------------+--------------------+-------+------+--------------------+
|485991194|2020-02-07 17:23:12|      view|  21400996|2232732082063278200|  electronics.clocks|  casio| 82.06|f039b328-dabb-444...|
|485991194|2020-02-07 21:30:52|      view|   4803977|2232732079706079299|       sport.bicycle|samsung|102.33|f039b328-dabb-444...|
|496765250|2020-02-24 19:23:42|      view|   5701037|2053013554415534427|electronics.video.tv|   swat|100.39|21467349-aaca-49b...|
|496765250|2020-02-24 19:24:51|      view|   5701037|2053013554415534427|electronics.video.tv|   swat|100.39|21467349-aaca-49b...|
|496765250|2020-02-24 19:25:14|      view|   5700793|2053013554415534427|el

In [9]:
%%time
month_01_filtered.write.mode("overwrite").parquet("./processed_data/month_01_filtered.parquet")

CPU times: user 9.51 ms, sys: 48.4 ms, total: 57.9 ms
Wall time: 3min 56s


In [10]:
# %%time
# month_02_filtered.write.mode("overwrite").parquet("./processed_data/month_02_filtered.parquet")